##### Copyright &copy; 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow データ検証

***TensorFlow Extended の主要コンポーネントの例***

注意：この例は、Jupyter スタイルのノートブックで今すぐ実行できます。セットアップは必要ありません。「Google Colab で実行」をクリックするだけです。

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/data_validation/tfdv_basic"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/data_validation/tfdv_basic.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/data_validation/tfdv_basic.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
</table></div>

このサンプルの colab ノートブックは、TensorFlow Data Validation (TFDV) を使用してデータセットを調査および視覚化する方法を示しています。これには、記述統計の確認、スキーマの推測、異常のチェックと修正、データセットのドリフトとスキューのチェックが含まれています。実稼働環境のパイプラインでデータセットが時間の経過とともにどのように変化するかなど、データセットの特性を理解することが重要です。また、データの異常を探し、トレーニング、評価、およびサービングデータセットを比較して、それらが一貫していることを確認することも重要です。

シカゴ市からリリースされた[タクシー乗車データセット](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)のデータを使用します。

注意：このWeb サイトは、シカゴ市の公式 Web サイト www.cityofchicago.org で公開されたデータを変更して使用するアプリケーションを提供します。シカゴ市は、この Web サイトで提供されるデータの内容、正確性、適時性、または完全性について一切の表明を行いません。この Web サイトで提供されるデータは、随時変更される可能性があります。かかる Web サイトで提供されるデータはユーザーの自己責任で利用されるものとします。

データセットの詳細については、[Google BigQuery](https://cloud.google.com/bigquery/) を[参照](https://cloud.google.com/bigquery/public-data/chicago-taxi)してください。[BigQuery UI](https://bigquery.cloud.google.com/dataset/bigquery-public-data:chicago_taxi_trips) でデータセット全体をご確認ください。

キーポイント：モデラーおよび開発者の皆さんは、このデータがどのように使用されるか、モデルの予測が引き起こす可能性のある潜在的メリット・デメリットについて考えてください。このようなモデルは、社会的バイアスと格差を拡大する可能性があります。特徴は解決しようとする問題に関連していますか、それともバイアスを導入しますか？ 詳細については、[機械学習における公平性](https://developers.google.com/machine-learning/fairness-overview/)についてご一読ください。

データセットの列は次のとおりです。

<table>
<tr>
<td>pickup_community_area</td>
<td>fare</td>
<td>trip_start_month</td>
</tr>
<tr>
<td>trip_start_hour</td>
<td>trip_start_day</td>
<td>trip_start_timestamp</td>
</tr>
<tr>
<td>pickup_latitude</td>
<td>pickup_longitude</td>
<td>dropoff_latitude</td>
</tr>
<tr>
<td>dropoff_longitude</td>
<td>trip_miles</td>
<td>pickup_census_tract</td>
</tr>
<tr>
<td>dropoff_census_tract</td>
<td>payment_type</td>
<td>company</td>
</tr>
<tr>
<td>trip_seconds</td>
<td>dropoff_community_area</td>
<td>tips</td>
</tr>
</table>

### Pip のアップグレード

ローカルで実行する場合にシステム Pip をアップグレードしないようにするには、Colab で実行していることを確認してください。もちろん、ローカルシステムは個別にアップグレードできます。

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### TensorFlow のインストール

**注意：Google Colab では、パッケージが更新されるため、このセルを初めて実行するときに、ランタイムを再起動（[ランタイム] &gt; [ランタイムの再起動...]）する必要があります。**

In [ ]:
!pip install tensorflow==2.2.0

## Python バージョンのチェック

In [ ]:
import sys

# Confirm that we're using Python 3
assert sys.version_info.major is 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

## TFDV のインストール

これにより、すべての依存関係が取得されます。これには1分かかります。互換性のない依存関係バージョンに関する警告またはエラーは無視します。

**注意：Google Colab では、パッケージが更新されるため、このセルを初めて実行するときに、ランタイムを再起動（[ランタイム] &gt; [ランタイムの再起動...]）する必要があります。**

In [ ]:
import tensorflow as tf

print('Installing TensorFlow Data Validation')
!pip install -q tensorflow_data_validation[visualization]

## ランタイムを再起動しましたか？

Google Colab を使用している場合は、上記のセルを初めて実行するときにランタイムを再起動（[ランタイム]　&gt; [ランタイムの再起動...]）する必要があります。これは、Colab がパッケージを読み込むために必要です。

## ファイルを読み込む

Google Cloud Storage からデータセットをダウンロードします。

In [ ]:
import os
import tempfile, urllib, zipfile

# Set up some globals for our file paths
BASE_DIR = tempfile.mkdtemp()
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'chicago_taxi_output')
TRAIN_DATA = os.path.join(DATA_DIR, 'train', 'data.csv')
EVAL_DATA = os.path.join(DATA_DIR, 'eval', 'data.csv')
SERVING_DATA = os.path.join(DATA_DIR, 'serving', 'data.csv')

# Download the zip file from GCP and unzip it
zip, headers = urllib.request.urlretrieve('https://storage.googleapis.com/artifacts.tfx-oss-public.appspot.com/datasets/chicago_data.zip')
zipfile.ZipFile(zip).extractall(BASE_DIR)
zipfile.ZipFile(zip).close()

print("Here's what we downloaded:")
!ls -R {os.path.join(BASE_DIR, 'data')}

### バージョンのチェック

In [ ]:
import tensorflow_data_validation as tfdv
print('TFDV version: {}'.format(tfdv.version.__version__))

## 統計を計算し、視覚化する

まず、[`tfdv.generate_statistics_from_csv`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/generate_statistics_from_csv)を使用して、トレーニングデータの統計を計算します。（警告は無視します）

TFDV は、[記述統計](https://github.com/tensorflow/metadata/blob/v0.6.0/tensorflow_metadata/proto/v0/statistics.proto)を計算し、存在する特徴やそれらの値分布の形などを含むデータの概要を迅速に提供します。

内部的には、TFDV は[Apache Beam](https://beam.apache.org/)のデータ並列処理フレームワークを使用して、大規模なデータセットの統計計算をスケーリングします。アプリケーションを TFDV とより深く統合させるには（データ生成パイプラインの最後に統計生成をアタッチする場合など）、API は統計生成用の Beam PTransform も公開します。

In [ ]:
train_stats = tfdv.generate_statistics_from_csv(data_location=TRAIN_DATA)

次に、[`tfdv.visualize_statistics`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/visualize_statistics)を使用します。これは、[ファセット](https://pair-code.github.io/facets/)を使用して、トレーニングデータの簡潔な視覚化を作成します。

- 数値の特徴とカテゴリの特徴が別々に視覚化され、各特徴の分布を示すグラフが表示されます。
- 値が欠落しているかゼロの特徴は、それらの特徴の例に問題がある可能性があることを視覚的に示すために、パーセンテージが赤で表示されることに注意してください。パーセンテージは、その特徴の値が欠落しているかゼロである例のパーセンテージです。
- `pickup_census_tract`の値を持つ例がないことに注意してください。これは次元削減の機会です。
- グラフの上にある[展開]をクリックして、表示を変更してみてください
- グラフのバーにカーソルを合わせて、バケットの範囲とカウントを表示してみてください
- 対数目盛と線形目盛を切り替えてみてください。対数目盛が`payment_type`カテゴリカル特徴の詳細をどのように示しているかに注目してください。
- [表示するグラフ]メニューから[分位数]を選択し、マーカーにカーソルを合わせて分位数のパーセンテージを表示してみてください

In [ ]:
tfdv.visualize_statistics(train_stats)

## スキーマを推測する

次に、[`tfdv.infer_schema`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/infer_schema)を使用してデータのスキーマを作成しましょう。<br>スキーマは、機械学習に関連するデータの制約を定義します。制約の例には、各特徴のデータ型（数値、または、カテゴリ）、またはデータ内に存在する頻度が含まれます。カテゴリカル特徴の場合、スキーマはドメイン（許容値のリスト）も定義します。スキーマの作成は、特に多くの特徴を備えたデータセットの場合、手間のかかる作業になる可能性があるため、TFDV は、記述統計に基づいてスキーマの初期バージョンを生成する方法を提供します。

実稼働環境のパイプラインの残りの部分は、TFDV が生成するスキーマが正しいことに依存するため、スキーマを正しく作成することが重要です。スキーマはデータのドキュメントも提供するため、異なる開発者が同じデータで作業する場合に役立ちます。[`tfdv.display_schema`](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/display_schema)を使用して、推測されたスキーマを表示し、確認します。

In [ ]:
schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema=schema)

## 評価データにエラーがないか確認します

これまでは、トレーニングデータのみを見てきましたが、評価データがトレーニングデータと一致していること（同じスキーマを使用していることなど）を確認することが重要です。また、評価データに、トレーニングデータとほぼ同じ数値の範囲の特徴の例が含まれていることも重要です。カテゴリカル特徴でも同じです。そうでない場合、損失面の一部を評価しなかったため、評価中に特定されないトレーニングの問題が発生する可能性があります。

- 各特徴には、トレーニングデータセットと評価データセットの両方の統計が含まれていることに注意してください。
- チャートにトレーニングデータセットと評価データセットの両方がオーバーレイされ、それらを簡単に比較できるようになっています。
- チャートにはパーセンテージビューが含まれています。これは、ログまたはデフォルトの線形スケールと組み合わせることができます。
- `trip_miles`の平均と中央値は、トレーニングデータセットと評価データセットで異なることに注意してください。これは問題を引き起こすでしょうか？
- 最大`tips`は、トレーニングデータセットと評価データセットで大きく異なります。これは問題を引き起こすでしょうか？
- 数値特徴チャートの展開をクリックし、対数スケールを選択します。`trip_seconds`特徴を確認し、最大値の違いに注目してください。評価は損失面の一部を見逃すでしょうか？

In [ ]:
# Compute stats for evaluation data
eval_stats = tfdv.generate_statistics_from_csv(data_location=EVAL_DATA)

# Compare evaluation data with training data
tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

## 評価の異常をチェックします

評価データセットは、トレーニングデータセットのスキーマと一致していますか？これは、許容値の範囲を特定するカテゴリカル特徴にとって特に重要です。

キーポイント：トレーニングデータセットにないカテゴリカル特徴値を持つデータを使用して評価しようとするとどうなるでしょうか？トレーニングデータセットの範囲外の数値特徴はどうなるでしょうか？

In [ ]:
# Check eval data for errors by validating the eval data stats using the previously inferred schema.
anomalies = tfdv.validate_statistics(statistics=eval_stats, schema=schema)
tfdv.display_anomalies(anomalies)

## スキーマの評価の異常を修正する

評価データには`company`の新しい値がいくつかありますが、トレーニングデータにはありません。また、`payment_type`の新しい値もあります。これらは異常と見なす必要がありますが、それらに対して何をするかは、データに関するドメイン知識によって異なります。異常が本当にデータエラーを示している場合は、基になるデータを修正する必要があります。それ以外の場合は、スキーマを更新して、評価データセットに値を含めることができます。

キーポイント：これらの問題を修正しなかった場合、評価結果にどのような影響があるでしょうか？

評価データセットを変更しない限り、すべてを修正することはできませんが、受け入れやすいスキーマ内のものを修正することはできます。例えば、特定の特徴の異常とは何かという見方を見直したり、カテゴリカル特徴の欠落値を含めるようにスキーマを更新したりできます。TFDV を利用することにより、修正が必要なものを見つけることができます。

これらの修正を行ってから、もう一度確認します。

In [ ]:
# Relax the minimum fraction of values that must come from the domain for feature company.
company = tfdv.get_feature(schema, 'company')
company.distribution_constraints.min_domain_mass = 0.9

# Add new value to the domain of feature payment_type.
payment_type_domain = tfdv.get_domain(schema, 'payment_type')
payment_type_domain.value.append('Prcard')

# Validate eval stats after updating the schema 
updated_anomalies = tfdv.validate_statistics(eval_stats, schema)
tfdv.display_anomalies(updated_anomalies)

TFDV を使用してトレーニング データと評価データが一致することを確認しました。

## スキーマの環境変数

また、この例では「サービング」データセットを分割しているので、それも確認する必要があります。デフォルトでは、パイプライン内のすべてのデータセットは同じスキーマを使用する必要がありますが、多くの場合、例外があります。たとえば、教師あり学習では、データセットにラベルを含める必要がありますが、推論用のモデルを提供する場合、ラベルは含まれません。場合によっては、スキーマをわずかに変更する必要があります。

**環境**を使用して、このような要件を表すことができます。特に、スキーマの特徴は、`default_environment`、`in_environment`、および、`not_in_environment`を使用して一連の環境に関連付けることができます。

たとえば、このデータセットでは、特徴`Tips`がトレーニングのラベルとして含まれていますが、サービングデータにありません。環境を指定しないと、異常として表示されます。

In [ ]:
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

以下では特徴`tips`について説明します。また、スキーマは乗車期間（秒）として浮動小数点数型の値を期待していましたが整数型の値があります。TFDV は、その違いを見つけ、生成されるトレーニングテータとサービングデータの不整合を明らかにします。モデルのパフォーマンスが（時には破壊的に）低下するまで、このような問題に気付かないことがよくあります。これが重大な問題であってもなくても、調査する必要があります。

この場合、整数値を浮上小数点に安全に変換できます。以下のとおり、スキーマを使用して型を推測するように TFDV に指示します。

In [ ]:
options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)
serving_stats = tfdv.generate_statistics_from_csv(SERVING_DATA, stats_options=options)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

これで、特徴`tips`（ラベル）が異常として表示されます（「列がドロップされました」）。サービングデータにラベルが含まれることは想定されていないため、TFDV にそれを無視するように指示します。

In [ ]:
# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Specify that 'tips' feature is not in SERVING environment.
tfdv.get_feature(schema, 'tips').not_in_environment.append('SERVING')

serving_anomalies_with_env = tfdv.validate_statistics(
    serving_stats, schema, environment='SERVING')

tfdv.display_anomalies(serving_anomalies_with_env)

## ドリフトとスキューのチェック

TFDV は、データセットがスキーマで設定された期待値に準拠しているかどうかを確認する他、リフトとスキューを検出する機能も提供します。TFDV は、スキーマで指定されたドリフト/スキューコンパレータに基づいてさまざまなデータセットの統計を比較することにより、このチェックを実行します。

### ドリフト

ドリフト検知は、カテゴリカルな特徴量で、連続したスパン (言い換えるとスパン N とスパン N+1 ) のデータ、例えば異なる日付の訓練データについてサポートしています。ここで、ドリフトは[L-無限大 距離](https://en.wikipedia.org/wiki/Chebyshev_distance)に基いて表されます。また、ドリフトが許容可能でないほど高い値をとった場合に警告を受け取るように、距離のしきい値を設定できます。正しく距離を設定することは、典型的にはドメイン知識や試行錯誤が必要な反復的なプロセスになります。

### 歪度

TFDV は、データ内の3種類の歪度（スキーマ歪度、特徴歪度、および分布歪度）を検出します。

#### スキーマ歪度

スキーマ歪度は、トレーニングデータとサービングデータが同じスキーマに準拠していない場合に発生します。トレーニングデータとサービングデータの両方が同じスキーマに準拠することが期待されます。 これらの間に予想される偏差（ラベル機能はトレーニングデータにのみ存在し、サービングには存在しないなど）は、スキーマの環境フィールドで指定する必要があります。

#### 特徴量の歪度

特徴量の歪度は、モデルがトレーニングする特徴値が、サービング時に表示される特徴値と異なる場合に発生します。たとえば、これは次の場合に発生する可能性があります。

- 特徴量を生成するデータソースがトレーニング時から実稼働環境に移行する間に修正される場合
- トレーニング時と実稼働環境とで特徴量を生成するロジックが一貫していない場合。例えば、何らかの変換処理をどちらか一方のコードにしか追加していない場合。

#### 分布歪度

分布の偏りは特徴量の分布が実稼働環境のデータの分布と著しく異なるときに生じます。分布の偏りが生じる主要な原因の1つは、トレーニングデータセットを生成するために異なるコードまたは異なるデータソースを使用することです。もう1つの理由は、サンプリングメカニズムの欠陥で、代表的でないサービングデータのサブサンプルがトレーニングされる場合です。

In [ ]:
# Add skew comparator for 'payment_type' feature.
payment_type = tfdv.get_feature(schema, 'payment_type')
payment_type.skew_comparator.infinity_norm.threshold = 0.01

# Add drift comparator for 'company' feature.
company=tfdv.get_feature(schema, 'company')
company.drift_comparator.infinity_norm.threshold = 0.001

skew_anomalies = tfdv.validate_statistics(train_stats, schema,
                                          previous_statistics=eval_stats,
                                          serving_statistics=serving_stats)

tfdv.display_anomalies(skew_anomalies)

この例では、多少のドリフトが見られますが、設定したしきい値をはるかに下回っています。

## スキーマの凍結

スキーマがレビューおよびキュレートされたので、「凍結」状態を反映するようにスキーマをファイルに保存します。

In [ ]:
from tensorflow.python.lib.io import file_io
from google.protobuf import text_format

file_io.recursive_create_dir(OUTPUT_DIR)
schema_file = os.path.join(OUTPUT_DIR, 'schema.pbtxt')
tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

## TFDV を使用する場合

ここでは、TFDV はトレーニングパイプラインの開始にのみ適用されましたが、実際には多くの用途があります。例を以下に示します。

- 推論のための新しいデータを検証して、急に不良な特徴を受け取り始めていないことを確認します。
- 推論のために新しいデータを検証して、モデルが決定面のその部分でトレーニングされていることを確認します。
- データを変換して特徴量エンジニアリング（おそらく[TensorFlow Transform](https://www.tensorflow.org/tfx/transform/) を使用）を行った後にデータを検証して、問題がないことを確認します。